### Imports

In [2]:
import os
import numpy as np
from PIL import Image
import torch
from scipy.spatial.distance import cdist
import pickle
import glob
import tifffile
import matplotlib.pyplot as plt
import re
# import cv2 
import rasterio

### Convert AMD to 16bit

In [ ]:
input_path = 'AMD/Abs Label Images'
output_path = 'AMD/Abs Label Images 16bit'

for filename in os.listdir(input_path):
    if filename.endswith('.tif') or filename.endswith('.tiff'):
        image_path = os.path.join(input_path, filename)

        image_array = tifffile.imread(image_path)

        dtype = image_array.dtype
        print(f"Processing {filename}")
        print(f"Image dtype: {dtype}")

        if dtype == np.uint8:
            bit_depth = 8
        elif dtype == np.uint16:
            bit_depth = 16
        elif dtype == np.float32:
            bit_depth = 32
        elif dtype == np.float64:
            bit_depth = 64
        else:
            bit_depth = "Unknown"
        print(f"Image bit depth: {bit_depth}")

        if bit_depth != 16:
            max_val = np.max(image_array)
            if max_val == 0:
                print(f"Warning: {filename} contains only zero values. Skipping normalization.")
                image_16bit_array = np.zeros_like(image_array, dtype=np.uint16)
            else:
                image_16bit_array = (image_array / max_val * 65535).astype(np.uint16)
            print(f"Converted to 16-bit: {image_16bit_array.dtype}")
        else:
            image_16bit_array = image_array

        output_filename = os.path.splitext(filename)[0] + '.tif'
        output_file_path = os.path.join(output_path, output_filename)

        tifffile.imwrite(output_file_path, image_16bit_array)

        print(f"Saved 16-bit image to {output_file_path}")


# for filename in os.listdir(input_path):
#     if filename.endswith('.tif') or filename.endswith('.tiff'):
#         image_path = os.path.join(input_path + '/', filename)

#         image = Image.open(image_path)

#         mode = image.mode
#         print(f"Image mode: {mode}")

#         image_array = np.array(image)

#         dtype = image_array.dtype
#         print(f"Image dtype: {dtype}")

#         if dtype == np.uint8:
#             bit_depth = 8
#         elif dtype == np.uint16:
#             bit_depth = 16
#         elif dtype == np.float32:
#             bit_depth = 32
#         elif dtype == np.float64:
#             bit_depth = 64
#         else:
#             bit_depth = "Unknown"

#         print(f"Image bit depth: {bit_depth}")

#         image_32bit = Image.open(image_path)
#         image_array = np.array(image_32bit)
#         image_16bit_array = (image_array / np.max(image_array) * 65535).astype(np.uint16)
#         image_16bit = Image.fromarray(image_16bit_array)

#         print(f"New Image dtype: {image_16bit_array.dtype}")

#         output_filename = os.path.splitext(filename)[0] + '.tif'
                
#         image_16bit.save(output_path + '/' + output_filename)

Processing 1B_D77_n984.tif
Image dtype: uint8
Image bit depth: 8
Converted to 16-bit: uint16
Saved 16-bit image to AMD/Abs Label Images 16bit/1B_D77_n984.tif
Processing 1B_D76_n774.tif
Image dtype: uint8
Image bit depth: 8
Converted to 16-bit: uint16
Saved 16-bit image to AMD/Abs Label Images 16bit/1B_D76_n774.tif
Processing 1B_D76_n760.tif
Image dtype: uint8
Image bit depth: 8
Converted to 16-bit: uint16
Saved 16-bit image to AMD/Abs Label Images 16bit/1B_D76_n760.tif
Processing 1B_D75_n733.tif
Image dtype: uint8
Image bit depth: 8
Converted to 16-bit: uint16
Saved 16-bit image to AMD/Abs Label Images 16bit/1B_D75_n733.tif
Processing 1B_D77_n947.tif
Image dtype: uint8
Image bit depth: 8
Converted to 16-bit: uint16
Saved 16-bit image to AMD/Abs Label Images 16bit/1B_D77_n947.tif
Processing 1B_D75_n727.tif
Image dtype: uint8
Image bit depth: 8
Converted to 16-bit: uint16
Saved 16-bit image to AMD/Abs Label Images 16bit/1B_D75_n727.tif
Processing 1B_D77_n953.tif
Image dtype: uint8
Image 

### 16bit Sanity Check

In [ ]:
image_path ='AMD/Abs Images 16bit/AMD1_Clone-A_D75_n990_r179_c13.tif'
image = tifffile.imread(image_path)

# mode = image.mode
# print(f"Image mode: {mode}")

image_array = np.array(image)

dtype = image_array.dtype
print(f"Image dtype: {dtype}")

if dtype == np.uint8:
    bit_depth = 8
elif dtype == np.uint16:
    bit_depth = 16
elif dtype == np.float32:
    bit_depth = 32
elif dtype == np.float64:
    bit_depth = 64
else:
    bit_depth = "Unknown"

print(f"Image bit depth: {bit_depth}")

Image dtype: uint16
Image bit depth: 16


### Print Images

In [ ]:
image1_path = 'Notebook/AMD/Abs Images 16bit/1A_D75_n990.tif'
# image2_path = 'Notebook/AMD/Abs Label Images/AMD1_Clone-A_D75_n990_r179_c13.tiff'
image2_path = 'Notebook/AMD/Abs Test Images/AMD1_Clone-A_D75_n1022_r579_c1213.tif'
image1_path = 'Notebook/AMD/Abs Test Label Images/AMD1_Clone-A_D75_n1022_r601_c1201.tif'


# image1 = Image.open(image2_path)
image1 = tifffile.imread(image1_path)

# image2 = Image.open(image2_path)
image2 = tifffile.imread(image2_path)

image1 = np.array(image1)
# image1 = np.expand_dims(image1, axis=2)
image2 = np.array(image2)
# image2 = np.expand_dims(image2, axis=2)

# Display the training image
plt.figure(figsize=(6, 6))
plt.imshow(image1)
plt.title('Training Image')

# Display the label image
plt.figure(figsize=(6, 6))
plt.imshow(image2)  
plt.title('Segmentation Label')

### Print Unique

In [11]:
path = "Notebook/AMD/Abs Images 16bit"

files = glob.glob(os.path.join(path, "*.tif"))

# Extract the unique filenames, ignoring everything after the second underscore
unique_filenames = set()
for file in files:
    filename = os.path.basename(file)
    parts = filename.split("_")
    if len(parts) == 3:
        parts = parts[0] + parts[1]        
        unique_filenames.add("_".join(parts))

for filename in sorted(unique_filenames):
    print(filename)


1_A_D_7_5
1_B_D_7_3
1_B_D_7_4
1_B_D_7_5
1_B_D_7_6
1_B_D_7_7
2_B_D_7_3
3_B_D_7_3
3_B_D_7_4
3_C_D_7_5


### Count Matches

In [4]:
# Define your source directories
image_source_dir = 'Notebook/AMD/Abs Images 16bit'
label_source_dir = 'Notebook/AMD/Abs Label Images 16bit'

# Get the list of all files in the source directories
image_files = set(os.listdir(image_source_dir))
label_files = set(os.listdir(label_source_dir))

# Find the matches
matches = image_files.intersection(label_files)
# Print the number of matches
print(f'Number of matches: {len(matches)}')

# Find the files that don't match
non_matches = image_files.symmetric_difference(label_files)
# Print the names of the non-matching files
print('Non-matches: ', non_matches)

Number of matches: 1031
Non-matches:  {'1A_D75_n991.tif'}


### Change names

In [12]:
folder_path = 'Notebook/AMD/Abs Label Images' 

files = os.listdir(folder_path)

for file_name in files:
    if file_name.endswith('.tiff'):
        match = re.match(r"AMD(\d+)_Clone-(\w+)_D(\d+)_n(\d+)", file_name)
        if match:
            new_name = f"{match.group(1)}{match.group(2)}_D{match.group(3)}_n{match.group(4)}.tif"
            old_file = os.path.join(folder_path, file_name)
            new_file = os.path.join(folder_path, new_name)
            os.rename(old_file, new_file)
            print(f"Renamed: {file_name} -> {new_name}")
        else:
            print(f"Skipped: {file_name} (pattern not matched)")

print("Renaming completed.")

Renamed: AMD1_Clone-A_D75_n1048_r979_c2013.tiff -> 1A_D75_n1048.tif
Renamed: AMD3_Clone-B_D73_n424_r2691_c1197.tiff -> 3B_D73_n424.tif
Renamed: AMD1_Clone-B_D76_n798_r601_c1842.tiff -> 1B_D76_n798.tif
Renamed: AMD3_Clone-C_D75_n146_r415_c8.tiff -> 3C_D75_n146.tif
Renamed: AMD3_Clone-B_D74_n452_r428_c890.tiff -> 3B_D74_n452.tif
Renamed: AMD2_Clone-B_D73_n99_r2197_c807.tiff -> 2B_D73_n99.tif
Renamed: AMD1_Clone-B_D76_n819_r1001_c1642.tiff -> 1B_D76_n819.tif
Renamed: AMD1_Clone-B_D76_n824_r1201_c242.tiff -> 1B_D76_n824.tif
Renamed: AMD1_Clone-B_D75_n690_r1597_c1207.tiff -> 1B_D75_n690.tif
Renamed: AMD1_Clone-B_D74_n661_r2421_c2270.tiff -> 1B_D74_n661.tif
Renamed: AMD2_Clone-B_D73_n66_r1398_c1209.tiff -> 2B_D73_n66.tif
Renamed: AMD3_Clone-B_D73_n327_r892_c998.tiff -> 3B_D73_n327.tif
Renamed: AMD1_Clone-A_D75_n1105_r2378_c2012.tiff -> 1A_D75_n1105.tif
Renamed: AMD1_Clone-B_D76_n804_r801_c642.tiff -> 1B_D76_n804.tif
Renamed: AMD1_Clone-B_D76_n875_r2000_c842.tiff -> 1B_D76_n875.tif
Renamed: A

### Change Name for CellPose

In [ ]:
folder_path = 'QBAM/Images' 

files = os.listdir(folder_path)

for file_name in files:
    if file_name.endswith('.tif'):
        match = re.match(r"AMD(\d+)_Clone-(\w+)_D(\d+)_n(\d+)", file_name)
        if match:
            new_name = f"img_{match.group(4)}.tif"
            old_file = os.path.join(folder_path, file_name)
            new_file = os.path.join(folder_path, new_name)
            os.rename(old_file, new_file)
            print(f"Renamed: {file_name} -> {new_name}")
        else:
            print(f"Skipped: {file_name} (pattern not matched)")

print("Renaming Images completed.")




# folder_path2 = 'QBAM/Segmentations' 

# files = os.listdir(folder_path2)

# for file_name in files:
#     if file_name.endswith('.tif'):
#         match = re.match(r"(.+?)_(.+?)_(n)(\d+)\.ome\.tif", file_name)
#         if match:
#             new_name = f"img_{match.group(4)}_masks.tif"
#             old_file = os.path.join(folder_path2, file_name)
#             new_file = os.path.join(folder_path2, new_name)
#             os.rename(old_file, new_file)
#             print(f"Renamed: {file_name} -> {new_name}")
#         else:
#             print(f"Skipped: {file_name} (pattern not matched)")

# print("Renaming Labels completed.")

### Invert Labels

In [20]:
folder_path = 'Notebook/AMD/Abs Label Images'


for path in os.listdir(folder_path):
    file = os.path.join(folder_path, path)

    if os.path.isfile(file):
        if file.endswith(".tif"):
            image = tifffile.imread(file)
            pil_image = Image.fromarray(image)
            colors_count_list = pil_image.getcolors()
            # print(colors_count_list)
                
            if colors_count_list[0][0] > colors_count_list[1][0]:
                np_image = np.array(pil_image)




                for count, c_bgr in colors_count_list:
                    print('\tcolor {} appeared {} times'.format(c_bgr, count))

                np_image = np.where(np_image == 0, 1, np_image)  # Convert -1 to 0
                np_image = np.where(np_image == 255, 0, np_image)  # Convert 1 to -1
                np_image = np.where(np_image == 1, 255, np_image)  # Convert -1 to 1




                tifffile.imwrite(file, np_image)
                print("Converted: ", file)

	color 0 appeared 54512 times
	color 255 appeared 11024 times
Converted:  Notebook/AMD/Abs Label Images/1B_D77_n984.tif
	color 0 appeared 53738 times
	color 255 appeared 11798 times
Converted:  Notebook/AMD/Abs Label Images/1B_D76_n774.tif
	color 0 appeared 52893 times
	color 255 appeared 12643 times
Converted:  Notebook/AMD/Abs Label Images/1B_D76_n760.tif
	color 0 appeared 54044 times
	color 255 appeared 11492 times
Converted:  Notebook/AMD/Abs Label Images/1B_D75_n733.tif
	color 0 appeared 54039 times
	color 255 appeared 11497 times
Converted:  Notebook/AMD/Abs Label Images/1B_D77_n947.tif
	color 0 appeared 53957 times
	color 255 appeared 11579 times
Converted:  Notebook/AMD/Abs Label Images/1B_D75_n727.tif
	color 0 appeared 54039 times
	color 255 appeared 11497 times
Converted:  Notebook/AMD/Abs Label Images/1B_D77_n953.tif
	color 0 appeared 52289 times
	color 255 appeared 13247 times
Converted:  Notebook/AMD/Abs Label Images/3C_D75_n139.tif
	color 0 appeared 54823 times
	color 255

### Merge Train/Test

In [17]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define your source directories
image_source_dir = 'QBAM/Images'
label_source_dir = 'QBAM/Segmentations'

train_image_dir = 'CellPose QBAM/Train'
train_label_dir = 'CellPose QBAM/Train Labels'
test_image_dir = 'CellPose QBAM/Test'
test_label_dir = 'CellPose QBAM/Test Labels'

# Get the list of all files in the source directory
files = [f for f in os.listdir(image_source_dir) if f.endswith('.tif')]

# Split the files into training and testing sets
train_files, test_files = train_test_split(files, test_size=0.1)

# Move the files to the appropriate directories
for file in train_files:
    shutil.move(os.path.join(image_source_dir, file), os.path.join(train_image_dir, file))
    shutil.move(os.path.join(label_source_dir, file.replace('.tif', '_masks.tif')), os.path.join(train_label_dir, file.replace('.tif', '_masks.tif')))

for file in test_files:
    shutil.move(os.path.join(image_source_dir, file), os.path.join(test_image_dir, file))
    shutil.move(os.path.join(label_source_dir, file.replace('.tif', '_masks.tif')), os.path.join(test_label_dir, file.replace('.tif', '_masks.tif')))


### Convert to 16bit and Grayscale

In [ ]:
# # input_folder = 'AMD/Abs Images' 
# # output_file = 'AMD/Abs Images 16bit'

# # for filename in os.listdir(input_folder):
# #     if filename.endswith('.tif') or filename.endswith('.tiff'):
# #         image_path = os.path.join(input_folder + '/', filename)

# #         image_rgb = tifffile.imread(image_path)

# #         if image_rgb.ndim == 3 and image_rgb.shape[2] == 3:
# #             print(f"Processing {filename}")
# #             print(f"Image shape: {image_rgb.shape}, dtype: {image_rgb.dtype}")

# #             if image_rgb.dtype != np.uint16:
# #                 image_rgb = image_rgb.astype(np.float32) / np.max(image_rgb)
# #                 image_gray = np.dot(image_rgb[..., :3], [0.2989, 0.5870, 0.1140])
# #                 image_gray_16bit = (image_gray * 65535).astype(np.uint16)
# #             else:
# #                 image_gray_16bit = np.dot(image_rgb[..., :3], [0.2989, 0.5870, 0.1140]).astype(np.uint16)
# #                 print(f"Converted to 16-bit grayscale: dtype {image_gray_16bit.dtype}")

# #             tifffile.imwrite(output_file + '/' + filename, image_gray_16bit)
# #             print(f"Grayscale 16-bit image saved to {output_file}")



# Load the image
img = tifffile.imread('AMD3B_D74_n531.tif')
# Convert the image to grayscale
# This calculation is based on the human perception of brightness
gray_img = np.dot(img[...,:3], [0.2989, 0.5870, 0.1140])
# Save the grayscale image
tifffile.imsave('AMD3B_D74_n531_gray.tif', gray_img)



# input_file = "AMD3B_D74_n531.tif"

# if os.path.exists(input_file):
#     print(f"File exists: {input_file}")
# else:
#     print(f"File does not exist: {input_file}")

# tiff_array = tifffile.imread(input_file)

# print(f"Image shape: {tiff_array.shape}")

# if len(tiff_array.shape) == 3 and tiff_array.shape[2] == 3:
#     r, g, b = [Image.fromarray(tiff_array[:,:,i]) for i in range(3)]
#     r.save("red_channel.tiff")
#     g.save("green_channel.tiff")
#     b.save("blue_channel.tiff")
    
#     # Combine channels into grayscale and ensure proper scaling
#     grayscale = np.dot(tiff_array[...,:3], [0.2989, 0.5870, 0.1140])
#     grayscale = np.clip(grayscale, 0, 255).astype(np.uint16)
#     Image.fromarray(grayscale).save("grayscale.tiff")
    
#     print("RGB channels split and saved as separate TIFF files.")
#     print("Combined grayscale image saved as grayscale.tiff.")
# else:
#     print("The image does not appear to be a standard RGB TIFF. Please check the file format.")

UnidentifiedImageError: cannot identify image file '/Users/matt/Desktop/Lab/GNN Pipeline/Notebook/AMD3B_D74_n531.tif'

### Get DNN-S Majority Vote Segmentations

In [ ]:
folder_path = "test"

for folder in os.listdir(folder_path):
    if folder != '.DS_Store':
        for original_folder in ['Original_seg']:
            for filename in os.listdir(folder_path + '/' + folder):
                if filename.endswith('.tif') or filename.endswith('.tiff'):

                    with rasterio.open(folder_path + '/' + folder + '/' + filename) as src:
                        image_data = src.read(1)

                    image_data[image_data < 1.5] = 0

                    with rasterio.open(filename, 'w', **src.meta) as dst:
                        dst.write(image_data, 1)

### Check for NaN in CSVs

In [9]:
import os
import pandas as pd

def check_for_nan_in_csvs(directory):
    results = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path)
            if df.isnull().values.any():
                results.append((filename, True))
            else:
                results.append((filename, False))

    return results

directory_path = 'Notebook/Dataset/Train'

nan_check_results = check_for_nan_in_csvs(directory_path)

# Print the results
for filename, contains_nan in nan_check_results:
    if contains_nan:
        print(f"{filename} contains NaN values.")
    # else:
    #     print(f"{filename} does not contain NaN values.")


### Invert Seg Color

In [3]:
input_path = 'Demonstration/Cell Seg txt'
output_path = 'Demonstration/output'

for filename in os.listdir(input_path):
    if filename.endswith('.tif') or filename.endswith('.tiff'):
        with rasterio.open(input_path + '/' + filename) as file:
            image_data = file.read(1)
            print("Processing Image: ", filename)

            for i in range(image_data.shape[0]):
                for j in range(image_data.shape[1]):
                    if image_data[i, j] < 3.5:
                        image_data[i, j] = 255
                    elif image_data[i, j] > 3.5:
                        image_data[i, j] = 0
        
            with rasterio.open(output_path + '/' + filename, 'w', **file.meta) as dst:
                dst.write(image_data, 1)

/Users/matt/anaconda3/envs/SLab/lib/python3.9/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/Users/matt/anaconda3/envs/SLab/lib/python3.9/site-packages/rasterio/__init__.py:314: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  dataset = writer(


Processing Image:  1B_D75_n719.tif
Processing Image:  AMD1_Clone-B_D75_n743_r2396_c2007.tif
Processing Image:  AMD1_Clone-B_D75_n742_r2396_c1807.tif
Processing Image:  AMD1_Clone-B_D75_n733_r2196_c2407.tif
